Дописал класс `Node` так, чтобы рассчет глубины происходил сам: 
- при добавлении листа к листу (вызова метода-`setter` либо `left()`, либо `right()`);
- при поворотах вокруг корня поддерева (вызов `reset_depth()`);

Плюс добавил `ldepth`, `rdepth`, `isleaf`, `__repr__` для удобства.

`balance factor (bf)` можно просто рассчитывать, он в любом случае постоянно меняется, глубину легче считать.

In [215]:
class Node:
    def __init__(self, data) -> None:
        self.data = data
        self._left = None
        self._right = None
        self.depth = 1

    @property
    def left(self):
        return self._left

    @property
    def right(self):
        return self._right

    def isleaf(self,):
        return self._left is None and self._right is None

    @left.setter
    def left(self, value):
        if self.isleaf():
            self._left = value
            self.depth += 1
        else:
            self._left = value

    @right.setter
    def right(self, value):
        if self.isleaf():
            self._right = value
            self.depth += 1
        else:
            self._right = value

    @property
    def ldepth(self):
        return self.left.depth if self.left else 0
    
    @property
    def rdepth(self):
        return self.right.depth if self.right else 0

    @property
    def bf(self,):
        return self.rdepth - self.ldepth
    
    def __repr__(self) -> str:
        return str((self.data, self.depth))
    
    def reset_depth(self):
        self.depth = max(self.ldepth, self.rdepth) + 1

Тут чисто механически все повороты, все предельно ясно + `recalc_depth()`, чтобы глубина считалась верно.

Верная глубина считаться будет только тогда, когда у левого и правого потомка будут посчитаны правильные глубины.

В целом, перерасчет глубины идет снизу вверх, по рекурсии, поэтому все должно быть хорошо.

In [216]:
def recalc_depth(*nodes):
    for node in nodes:
        node.reset_depth()

def left_turn_(p: Node):
    q = p.right

    p.right = q.left
    q.left = p

    recalc_depth(p, q)
    return q
    
def right_turn_(q: Node):
    p = q.left

    q.left = p.right
    p.right = q

    recalc_depth(p, q)
    return p

def big_left_turn_(p: Node):
    q = p.right
    s = q.left

    q.left = s.right
    p.right = s.left
    s.left = p
    s.right = q

    recalc_depth(p, q, s)
    return s
    
def big_right_turn_(p: Node):
    q = p.left
    s = q.right

    q.right = s.left
    p.left = s.right
    s.left = q
    s.right = p

    recalc_depth(p, q, s)
    return s

In [217]:
class Tree:
    def __init__(self) -> None:
        self.root = None
    
    def push(self, data):
        if self.root is None:
            self.root = Node(data)
        else:
            self.rec_push_(self.root, data)
            # Добавление балансировки
            if abs(self.root.bf) == 2:
                self.root = self.balance_(self.root)     

    def rec_push_(self, cursor, data):
        next_cur = None
        if cursor.data > data:
            if cursor.left is None:
                cursor.left = Node(data)
            else:
                next_cur = cursor.left
        else:
            if cursor.right is None:
                cursor.right = Node(data)
            else:
                next_cur = cursor.right
                
        if next_cur:
            self.rec_push_(next_cur, data)
            # Добавление балансировки
            if abs(next_cur.bf) == 2:
                next_cur = self.balance_(next_cur)
                if cursor.data > data:
                    cursor.left = next_cur
                else:
                    cursor.right = next_cur
            cursor.reset_depth()
            
    def balance_(self, cursor):
        turn = None
        if cursor.bf > 0:
            turn = left_turn_ if cursor.right.bf >= 0 else big_left_turn_
        else:
            turn = right_turn_ if cursor.left.bf <= 0 else big_right_turn_
        
        print(f'balance at {cursor.data} with {turn}')
        return turn(cursor)

    def __repr__(self,):
        res = []
        if self.root:
            self.repr_(self.root, 0, res)
        return '\n'.join(res) if res else str(())
    
    def repr_(self, cursor, depth, s):
        s.append(depth * '\t' + str((cursor.data, cursor.depth, cursor.bf)))
        if cursor.left:
            s.append(depth * '\t' + 'left:')
            self.repr_(cursor.left, depth + 1, s)
        if cursor.right:
            s.append(depth * '\t' + 'right:')
            self.repr_(cursor.right, depth + 1, s)

In [218]:
t = Tree()

for i in (10, 15, 11, 5, 7, 8):
    print(i)
    t.push(i)
    print(t)
    print(f'bf={t.root.bf}')
    print('-' * 30)


10
(10, 1, 0)
bf=0
------------------------------
15
(10, 2, 1)
right:
	(15, 1, 0)
bf=1
------------------------------
11
balance at 10 with <function big_left_turn_ at 0x00000274830D8EE0>
(11, 2, 0)
left:
	(10, 1, 0)
right:
	(15, 1, 0)
bf=0
------------------------------
5
(11, 3, -1)
left:
	(10, 2, -1)
	left:
		(5, 1, 0)
right:
	(15, 1, 0)
bf=-1
------------------------------
7
balance at 10 with <function big_right_turn_ at 0x00000274830D8DC0>
(11, 3, -1)
left:
	(7, 2, 0)
	left:
		(5, 1, 0)
	right:
		(10, 1, 0)
right:
	(15, 1, 0)
bf=-1
------------------------------
8
balance at 11 with <function big_right_turn_ at 0x00000274830D8DC0>
(10, 3, 0)
left:
	(7, 2, 0)
	left:
		(5, 1, 0)
	right:
		(8, 1, 0)
right:
	(11, 2, 1)
	right:
		(15, 1, 0)
bf=0
------------------------------


In [219]:
t

(10, 3, 0)
left:
	(7, 2, 0)
	left:
		(5, 1, 0)
	right:
		(8, 1, 0)
right:
	(11, 2, 1)
	right:
		(15, 1, 0)